In [1]:
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd

def extract_master(oneURL):
    return int(oneURL.split('/')[-1].split('-')[0])

def extract_type(oneURL):
    return oneURL.split('/')[-2]

def insert_master_list(master_list):
    """ insert multiple masters into the masters table  """
    sql = "INSERT INTO master_owned (master_id) VALUES %s"
    conn = None
    try:
        # connect to the PostgreSQL database
        conn = psycopg2.connect("dbname=discogs user=postgres password=postgres")
        # create a new cursor
        cur = conn.cursor()
        # make list of tuples
        arr = [(i,) for i in master_list]
        # execute the INSERT statement
        execute_values(cur, sql, arr)
        # commit the changes to the database
        conn.commit()
        # close communication with the database
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [2]:
myPath = 'C:\\Users\\mikejseay\\Documents\\Code\\discogs-xml2db\\my_collection.csv'

In [3]:
myDF = pd.read_csv(myPath)
myDF['masterNum'] = myDF['URL'].apply(extract_master)
myDF['type'] = myDF['URL'].apply(extract_type)
releaseNoMaster = myDF['type'] == 'release'
releaseNoMaster.sum()

8

In [4]:
myDF.drop(myDF.index[releaseNoMaster], inplace=True)

In [5]:
myDF.to_csv('my_collection_export.csv')

In [7]:
insert_master_list(myDF['masterNum'].values.tolist())